<a href="https://colab.research.google.com/github/carinunez/Labs-Grafos/blob/main/Lab_Graph_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio Graph Embeddings

En este laboratorio crearemos un grafo sintético y entrenaremos una GNN para predecir etiquetas.

Estas primeras celdas le permitirán verificar las versiones del software disponible en Colab e instalar las librerías faltantes.

In [ ]:
!python -c "import torch; print(torch.version.cuda)"

In [ ]:
!python -c "import torch; print(torch.__version__)"

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

In [ ]:
!pip install torch-sparse -f https://data.pyg.org/whl/torch-2.6.0+cu124.html

In [ ]:
!pip install torch-geometric

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import networkx as nx
import random
import numpy as np
from sklearn.cluster import KMeans

In [ ]:
# 1. Crear distintos tipos de grafo y asignar etiquetas
def spectral_labels(G, k=3):
    lap = nx.normalized_laplacian_matrix(G).todense()
    eigvecs = np.linalg.eigh(lap)[1][:, 1:k+1]
    return KMeans(n_clusters=k, random_state=42).fit_predict(eigvecs)

def generate_graph(graph_type='sbm', num_nodes=150):
    if graph_type == 'sbm':
        sizes = [num_nodes // 3] * 3
        probs = [[0.3, 0.05, 0.02], [0.05, 0.3, 0.04], [0.02, 0.04, 0.3]]
        G = nx.stochastic_block_model(sizes, probs, seed=42)
        labels = [i for i, size in enumerate(sizes) for _ in range(size)]
    elif graph_type == 'erdos':
        G = nx.erdos_renyi_graph(num_nodes, 0.05, seed=42)
        labels = spectral_labels(G)
    elif graph_type == 'barabasi':
        G = nx.barabasi_albert_graph(num_nodes, 2, seed=42)
        labels = spectral_labels(G)
    else:
        raise ValueError("Unknown graph type")

    G = max((G.subgraph(c) for c in nx.connected_components(G)), key=len)
    edge_index = torch.tensor(list(G.edges), dtype=torch.long).t().contiguous()
    features = torch.eye(G.number_of_nodes())
    labels = torch.tensor(labels[:G.number_of_nodes()], dtype=torch.long)
    return G, features, labels, edge_index

In [ ]:
# 2. GCN
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers=2):
        super().__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append("PRIMERA CAPA")
        for _ in range(num_layers - 2):
            self.convs.append("SIGUIENTES CAPAS")
        self.convs.append(GCNConv(hidden_channels, out_channels))

    def forward(self, x, edge_index):
        for conv in self.convs[:-1]:
            x = "ACTIVACION(APLICAR CAPA(...))"
        return self.convs[-1](x, edge_index)

In [ ]:
# 3. entrenamiento y test
def train(model, data, loss_fn):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    model.train()
    for epoch in range(101):
        out = model(data.x, data.edge_index)
        loss = loss_fn(out[data.train_mask], data.y[data.train_mask])
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        if epoch % 20 == 0:
            print(f"Epoch {epoch} | Loss: {loss.item():.4f}")

def test(model, data):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    acc = int((pred[data.test_mask] == data.y[data.test_mask]).sum()) / int(data.test_mask.sum())
    print(f"Test Accuracy: {acc:.4f}")


In [ ]:
# 4. Ejemplo de ejecución
# Elegir tipo de grafo: 'sbm', 'erdos', 'barabasi'
G, X, Y, E = generate_graph('barabasi')
n = X.size(0)
train_mask = torch.zeros(n, dtype=torch.bool); train_mask[:int(0.8*n)] = True
test_mask = ~train_mask
data = Data(x=X, edge_index=E, y=Y, train_mask=train_mask, test_mask=test_mask)

model = GCN(X.size(1), 16, 3, 2)
train(model, data, F.cross_entropy)
test(model, data)